<a href="https://colab.research.google.com/github/kwnstantinosRoumeliwtis/ML_course/blob/main/ML_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Άσκηση 8**

Επειδή είχα θέμα με το word αρχείο στο preview, που δεν με άφηνε να ανοίξω τα links για το βοηθητικό jupyter και μπόρεσα και άνοιξα μόνο από τον φάκελο old το αρχείο με το mnist dataset. Θα κανω την εργασία με tensorflow που έχω δουλέψει στο προπτυχιακό σαν φοιτητής.

1-2
Εδώ κατεβάζω το dataset μέσω της kagglehub. Επειδή έπαιξα με τα paths είδα ότι το dataset με τους φακέλους ανα κατηγορία είναι εμφολευμένο σε έναν φάκελο dataset οπότε με την βιβλιοθήκη os ενώνω το path.

In [15]:

import kagglehub
import tensorflow as tf
import os

path = kagglehub.dataset_download("khushikhushikhushi/dog-breed-image-dataset")
print("Path to dataset files:", path)
dataset_path = os.path.join(path, "dataset")

Using Colab cache for faster access to the 'dog-breed-image-dataset' dataset.
Path to dataset files: /kaggle/input/dog-breed-image-dataset


3
Σε αυτό το σημείο με τη βοήθεια της βιβλιοθήκης tensorflow φορτώνω το dataset και κάθε εικόνα φορτώνει σε μέγεθος 64 * 64 * 3, ζητάω από τη βιβλιοθήκη να ανακατέψει τις εικόνες και αρχικά δίνω batch size = 1 για να μπορέσω να διαχωρίσω train-test set. Αμέσως μετά κάνω unbatch και προσθέτω το dataset στην λίστα all_data. Κρατάω το 80% των δεδομενων για training και το υπόλοιπο για test(Αν δεν έκανα shuffle τότε το test θα γινόταν στις 2 τελευταίες κατηγορίες μόνο που δεν θα είχε εκπαιδευτεί το νευρωνικό δίκτυο). Ύστερα και για το train και για το test κάνω τα tensors batches με size 32. Τέλος με το rescaling πολλαπλασιάζω όλα τα pixels κάθε εικόνας με 1/255 και την κανονικοποιώ φέρνω τις εντάσεις στο εύρος 0-1. Οι επόμενες εντολές είναι καθαρά για την καλύτερη διαχείρηση της gpu και της μνήμης. * * AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:

full_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(64, 64),
    batch_size=1,
    shuffle=True,
    seed=42
)

all_data = list(full_ds.unbatch())
total = len(all_data)

train_size = int(0.8 * total)

train_data = all_data[:train_size]
test_data  = all_data[train_size:]

train_images = [x for x, y in train_data]
train_labels = [y for x, y in train_data]

test_images = [x for x, y in test_data]
test_labels = [y for x, y in test_data]

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)
test_ds  = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

print("Train batches:", train_ds.cardinality().numpy())
print("Test batches:", test_ds.cardinality().numpy())


Found 967 files belonging to 10 classes.
Train batches: 25
Test batches: 7


Εδώ δημιουργούμε το one hot vectors για κάθε κλάση. Έχουμε 10 κλάσεις οπότε έχουμε labels πχ [0,0,0,0,0,0,0,0,0,1] αν η εικόνα μας ανήκει στην 10η κατηγορία, επειδή έχουμε πολλές κλάσεις, όχι μόνο 2.

In [17]:
num_classes = 10

def one_hot(images, labels):
    labels = tf.one_hot(labels, depth=num_classes)
    return images, labels

train_ds = train_ds.map(one_hot)
test_ds  = test_ds.map(one_hot)

4-5 . Εδώ δημιουργούμε ένα MLP και του λέμε όλα sequential άρα κάθε νευρώνας ενός layer προωθεί το σήμα σε όλους του επόμενου layer. Λέμε ότι σαν είσοδο θα πάρει τις εικόνες μας σε tensor μεγέθους 64 * 64 * 3 (RGB). Το πρώτο layer έχει 512 νευρώνες, το δεύτερο 256 και το τρίτο 128. Κάθε νευρώνας χρησιμοποιεί συνάρτηση Relu για να ενεργοποιήσει ή όχι το σήμα του. Το τελευταίο layer εξόδου αποτελείται από 10 νευρώνες που έχουν συνάρτηση ενεργοποίησης softmax, δηλαδή στο one hot vector προβλέπεται ένα ποσοστό (πιθανότητα) για κάθε μία από τις 10 κλάσεις.

In [18]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Flatten(input_shape=(64, 64, 3)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Εδώ θέτουμε τις παραμέτρους για την εκπαίδευση

In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Εδώ εκπαιδεύουμε για 30 εποχές και εμφανίζουμε το accuracy και το loss ανά εποχή για τα training data.

In [20]:
history = model.fit(
    train_ds,
    epochs=30
)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1186 - loss: 5.9300
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2319 - loss: 2.1456
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3078 - loss: 2.1443
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4161 - loss: 1.7635
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4996 - loss: 1.4991
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5877 - loss: 1.2978
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6451 - loss: 1.0695
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7658 - loss: 0.7518
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8301 - loss: 0.6117
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8736 - loss: 0.5011
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8873 - loss: 0.4297
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8939 - l

Εδώ κάνουμε predict στα test δεδομένα και εμφανίζουμε accuracy και loss.

In [21]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 1.0000 - loss: 0.0209
Test accuracy: 1.0


7
 Εδώ δημιουργούμε ένα cnn στην είσοδο έχουμε 32 filters μεγέθους 3*3, στο επόμενο layer 64 kernels, στο επόμενο 128 και στο τέλος προσθέτουμε ένα μη cnn layer το οποίο κάνει την δουλειά της ταξινόμησης αφού εξάγαμε τα τοπικά χαρακτηριστικά από τα cnn layers. Ύστερα η προώθηση γίνεται στο layer εξόδου το οποίο αποτελείται από 10 νευρώνες και χρησιμοποιεί softmax σε αντίθεση με όλους τους νευρώνες των προηγούμενων layers πoυ χρησιμοποιούσαν ReLU. Εδώ να σημειώσουμε ότι μετά από κάθε cnn layer κάναμε max pooling. Το καλύτερο μοντέλο είναι το CNN διότι έχει μεγαλύτερη ακρίβεια και πιο γρήγορη σύγκλιση. Αυτό γίνεται γιατί το CNN με τα kernels εξάγει τα τοπικά χαρακτηριστικά των εικόνων και δίνει για ταξινόμηση καλύτερα features στο dense layer. Ανιχνεύει κρυφά χαρακτηριστικά, ακμές γεγονός που βοηθάει στην ταξινόμηση των κατηγοριών.

In [22]:

modelcnn = models.Sequential([
    layers.Input(shape=(64, 64, 3)),

    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [23]:
modelcnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [24]:
history = modelcnn.fit(
    train_ds,
    epochs=30
)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.1251 - loss: 2.3141
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1697 - loss: 2.2304
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3701 - loss: 1.7864
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6137 - loss: 1.1377
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8310 - loss: 0.5788
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9360 - loss: 0.2388
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9838 - loss: 0.0887
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9762 - loss: 0.0762
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9944 - loss: 0.0283
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9994 - loss: 0.0098
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0047
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - l

In [25]:
test_loss, test_acc = modelcnn.evaluate(test_ds)
print("Test accuracy:", test_acc)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 1.0000 - loss: 7.0340e-05
Test accuracy: 1.0


8)
Εδώ παρ'ότι η ακρίβεια είναι 100% θα εφαρμόσω batch normalization.

In [26]:
model_cnn_batch_normalization = models.Sequential([
    layers.Input(shape=(64, 64, 3)),

    layers.Conv2D(32, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(10, activation='softmax')
])

model_cnn_batch_normalization.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model_cnn_batch_normalization.fit(
    train_ds,
    epochs=30
)

test_loss, test_acc = model_cnn_batch_normalization.evaluate(test_ds)
print("Test accuracy:", test_acc)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 101ms/step - accuracy: 0.4554 - loss: 1.8622
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9947 - loss: 0.2040
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9981 - loss: 0.0543
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0230
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0151
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0113
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0087
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0068
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0060
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0053
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0048
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 